In [1]:
import pandas as pd
import os
import numpy as np
import re

In [2]:
path = "C:/Users/zkarimib@volvocars.com/OneDrive - Volvo Cars/Zohreh/Consultant Supplier Quality/Counsultant Supplier/Excel File/Company Excel LTI reports/All Consultant Company/"
input_path = path + "in" + "/"
output_path = path + "out" + "/"
filter_path = path + "filter" + "/"
print(input_path)

C:/Users/zkarimib@volvocars.com/OneDrive - Volvo Cars/Zohreh/Consultant Supplier Quality/Counsultant Supplier/Excel File/Company Excel LTI reports/All Consultant Company/in/


In [3]:
file_output_list = os.listdir(output_path)
print(file_output_list)

['Combine.xlsx', 'Combine1.xlsx', 'Combine3.xlsx', 'Company List.xlsx', 'statistical Methods.xlsx', 'statistical Methods2.xlsx']


In [4]:
file_filter_list=os.listdir(filter_path)
print(file_filter_list)

['automobile_manufacturers.xlsx', 'Automotive Companies.xlsx', 'Consulting Firms.xlsx', 'Top 50 stora arbestgivare.xlsx', 'Universum_Sweden_2023_ Young Profesisonals.xlsx']


In [5]:
# Load the Excel file
xls = pd.ExcelFile(output_path+'Combine.xlsx')

In [6]:
print(xls.sheet_names)

['Overview', 'Locations', 'Company Movements', 'Location Movements', 'Industry Movements', 'Titles', 'Skills', 'Attrition by Functions', 'Attrition by Locations', 'Schools', 'Degrees', 'Fields of Study']


In [7]:
# Specify the sheet name you're interested in
sheet_name = 'Company Movements'
company_movements=pd.read_excel(xls, sheet_name= sheet_name)

# Print the columns of the DataFrame
print(company_movements.columns.tolist())

['Company', 'Departures', 'Hires', 'Ratio', 'Net change', 'Consultant Company', 'Automotive Company', 'company_lower', 'Top 50 stora arbestgivare', 'Company_Name']


In [21]:
company_movements['Company'] = company_movements['Company'].str.lower()

In [22]:
# read the filter data excel file for consultancy and automotive company list

consultancies_list = pd.read_excel(filter_path + "Consulting Firms.xlsx")
automative_list = pd.read_excel(filter_path + "Automotive Companies.xlsx")

In [23]:
print(consultancies_list.columns.tolist())
print(automative_list.columns.tolist())

['Consulting Company', 'Primary Focus']
['Company', 'Industry']


In [24]:
# Escaping and lowercasing each company name for use in regular expressions, if necessary
automative_list['Company'] = automative_list['Company'].str.lower()
consultancies_list['Consulting Company'] = consultancies_list['Consulting Company'].str.lower()

print(automative_list.head()) 

                        Company         Industry
0                   volvo group  Automotive OEMS
1                  volvo trucks  Automotive OEMS
2  volvo construction equipment  Automotive OEMS
3                  scania group  Automotive OEMS
4                   volvo penta  Automotive OEMS


In [25]:
# Create a empty dataframe and store the company names in it
all_companies = pd.DataFrame()
all_companies['Company'] = company_movements['Company']
#all_companies

In [26]:
# merge the consultant company with all the company data
consultancy_companies = consultancies_list.merge(all_companies, left_on='Consulting Company', right_on='Company', how='inner')

# drop 'consulting company' column and remove duplicates from the dataframe
consultancy_companies = consultancy_companies.drop(columns=['Consulting Company']).drop_duplicates()

# add a new column to categorize them as consulting companies
consultancy_companies['Industry']='Consultancy'

# remove all the duplicates if any
consultancy_companies = consultancy_companies.drop_duplicates(keep='first')
consultancy_companies

,Primary Focus,Company,Industry
0,IT,diadrom,Consultancy
1,Engineering,infotiv,Consultancy
2,Engineering,semcon,Consultancy
9,IT,innofactor,Consultancy
10,MGMT,aptitud,Consultancy
11,IT,triona,Consultancy
12,Engineering,afry,Consultancy
33,Engineering,knightec,Consultancy
41,MGMT,cartina,Consultancy
42,IT,annevo,Consultancy


In [27]:
# combine both consultancy and automotive lists 
consultancy_automotive_combined = pd.concat([automative_list,consultancy_companies])
consultancy_automotive_combined = consultancy_automotive_combined.drop_duplicates(keep='first')
other_companies = all_companies[~all_companies['Company'].isin(consultancy_automotive_combined['Company'])].drop_duplicates()

In [28]:
# concatenate all the company lists (automotive, consulatncy and other)
result = pd.concat([consultancy_automotive_combined,other_companies])
# if the company is not an automotive or consultancy then list it as 'Other'
result['Industry'] = result['Industry'].fillna('Other')
result.drop_duplicates()

,Company,Industry,Primary Focus
0,volvo group,Automotive OEMS,NaN
1,volvo trucks,Automotive OEMS,NaN
2,volvo construction equipment,Automotive OEMS,NaN
3,scania group,Automotive OEMS,NaN
4,volvo penta,Automotive OEMS,NaN
...,...,...,...
2314,professionals\ nord,Other,NaN
2319,schibsted,Other,NaN
2320,sigma\ connectivity,Other,NaN
2322,single\ technologies,Other,NaN


In [29]:
# Export the data into an excel file
result.to_excel(output_path+'Company List.xlsx',sheet_name='Company List',index=False)